# Authors & References:

This notebook is created by:
- Joshua Adrian Cahyono
- Clayton Fernalo
- Bryan Atista Kiely

### Dataset Reference:

Guo, S., Lin, Y., Feng, N., Song, C., & Wan, H. (2019). Attention Based Spatial-Temporal Graph Convolutional Networks for Traffic Flow Forecasting. Proceedings of the AAAI Conference on Artificial Intelligence, 33(01), 922-929. https://doi.org/10.1609/aaai.v33i01.3301922

### Additional Reference:

ChatGPT is used for debugging, clarification of concepts, and the writing of comments and descriptions. 

<a id="contents"></a>
# Table of Content
1. [Problem Introduction](#section-1)
2. [Import Libraries & Initialize Plots](#section-2)
3. [Data Formatting](#section-3)
4. [Data Visualization & EDA](#section-4)
5. [Data Preparation](#section-5)
6. [Train Model](#section-6)
7. [Evaluate Prediction](#section-7)
8. [Conclusion](#section-8)

<a id="section-1"></a>
# Problem Introduction

We have found an interesting traffic dataset `PEMS-08 Dataset`, which contains the traffic data in San Bernardino from July to August in 2016. There are 170 locations with detectors recording every 5 minute intervals of traffic information. The dataset includes 3 features: `flow`, `occupy`, `speed`. The details of the features are as the following:

- The flow variable in the PEMS08 dataset represents the number of vehicles that pass through the loop detector per time interval (5 minutes in this case). It is measured in vehicles per 5-minute interval.

- The occupancy variable represents the proportion of time during the time interval (5 minutes) that the detector was occupied by a vehicle. It is measured as a percentage.

- The speed variable represents the average speed of the vehicles passing through the loop detector during the time interval (5 minutes). It is measured in miles per hour (mph).
 
From this, we thought of an interesting question: what if we create an model that can predict and properly rank the locations with the most traffic? We believe that this predictive task could potentially help the police in choosing the best locations to patrol given a particular time. To simply the problem, we believe that `occupy` encaptures the estimated traffic at a certain time, so we will be using `occupy` as our target variable.

Formally, we formulated the following problem statement:
For $N$ prediction of the `occupy` variable and $K$ locations, make a predictive model to minimize $\sum_{t=0}^{N-1} \rho_t$, where $\rho_t$ is defined as the [Spearman correlation](https://en.wikipedia.org/wiki/Spearman%27s_rank_correlation_coefficient) of the prediction values and the actual values.

The Spearman correlation coefficient is a measure of the strength and direction of the monotonic relationship between two variables. It is not calculated based on the absolute value, but rather it only considers the relativity of the ranks of the variables. This is perfect for our task, since we want to focus on patrol location optimization.

Our current strategy is to first focus on minimizing `RMSE` on predicting the actual values of the target variable `occupy`, and to simply use the rank of the values. For this, we will train an `LSTM` model and use the moving average of previous `occupy` values as the baseline.

<a id="section-2"></a>
# Import Libraries & Initialize Plots

As always, we will first import all the libraries necessary for this notebook to run. We put all the imports at the top to make it easier to check the dependencies.

In [ ]:
# Common libraries for data cleaning and visualization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from numpy import load # use to load an npz file
from scipy.signal import periodogram # use to graph a periodogram to get seasonality analysis
from sklearn.preprocessing import MinMaxScaler # use to normalize the data features

# keras library to create NN models
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

# libraries for the metrics we will use (RMSE and Spearman)
from keras.metrics import RootMeanSquaredError
import scipy.stats as stats

In addition, we changed the notebook default plotting settings to make the graph prettier.

In [ ]:
# Set plot settings
plt.rcParams.update({'font.size': 12, 'font.family': 'serif'})
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['figure.dpi'] = 100
plt.rcParams['axes.grid'] = True
plt.rcParams['axes.grid.which'] = 'both'
plt.rcParams['grid.alpha'] = 0.5

<a id="section-3"></a>
# Data Formatting

Interestingly, the data is in the format of npz. npz is a file format used by the NumPy library to store arrays and metadata in a compressed format. We will first convert it to pandas dataframe, since we are more familiar with it.

In [ ]:
data = load('/kaggle/input/traffic-flow/pems08.npz')

In [ ]:
lst = data.files
print(data[lst[0]].shape)
print(data[lst[0]])

The file gives a numpy array with dimensions (17856, 170, 3), which are in the form of (timesteps, location, features). For the 3 features, the order is `flow`,`occupy`,`speed`.

In [ ]:
traffic_data = data[lst[0]]
data_dict = []
# loop for every timestep and every location and add as a single row
for timestep in range(traffic_data.shape[0]):
    for location in range(traffic_data.shape[1]):
        data_dict.append({
            "timestep" : timestep+1,
            "location" : location,
            "flow"     : traffic_data[timestep][location][0],
            "occupy"   : traffic_data[timestep][location][1],
            "speed"    : traffic_data[timestep][location][2]
        })

We changed it into a list of dictionary. Each row represents a single timestep in a single location.

In [ ]:
df = pd.DataFrame(data_dict)
df.to_csv("traffic.csv", index=False)

Finally, we save it into a csv file called `traffic.csv`. So, if you already have the file `traffic.csv`, you won't need to run the code above.

<a id="section-4"></a>
# Data Visualization & EDA

Now it is time for Data Visualization and Exploratory Data Analysis. Here, we will see through the data and clean and apply feature engineering when necessary.

In [ ]:
traffic = pd.read_csv("/kaggle/working/traffic.csv")
print(len(traffic))
traffic.head()

The data consist of $17856 \cdot 170 = 3035520$ rows. That is a lot of rows! We will have to handle that later, but for now, let us look into the data.

In [ ]:
traffic.info()

All of the data seem to be either integer or float, nothing out of the ordinarily here. `flow`, `occupy` and `speed` are numerical and continuous data.

In [ ]:
traffic.count()

In [ ]:
traffic.isna().sum()

Also nothing surprising here, and fortunately the data has no null values, so we don't need to impute any values.

In [ ]:
traffic.describe()

In [ ]:
traffic['flow'].hist(bins=100);

In [ ]:
traffic['occupy'].hist(bins=100);

In [ ]:
traffic['speed'].hist(bins=100);

Looking at the distribution of `flow`, `occupy` and `speed`, it looks like they are spread out nicely and roughly follows a nice (a bit skewed) normal distribution. However, one thing to note is that the order of magnitude of the values are quite different, so we should normalize and scale the data later.

Now, we would like to explore the data further. For this, we will choose a single random location.

In [ ]:
location_0 = traffic[traffic["location"]==50].reset_index()
location_0.head()

In [ ]:
location_0["occupy"][:1000].plot()

In [ ]:
location_0["flow"][:1000].plot()

In [ ]:
location_0["speed"][:1000].plot();

We limit the plot to the first 1000 values since there are too many timesteps and the plot will be hard to look and analyze. From the plot above (and checking some other random locations), there seem to be a strong seasonality on `flow` and more importantly `occupy` , which is out target variable. 

Of course in any time series prediction, lag features are used as the main features. Specifically, $k$ lag features means that we use the last $k$ values to predict the next value. So, it is a good measure to check the correlation of present $(t_n)$ and future $(t_{n+1})$ values

In [ ]:
COR_STEP = 1
pres = traffic[['flow', 'occupy', 'speed']][0:-(COR_STEP)].reset_index(drop=True)
future = traffic[['flow', 'occupy', 'speed']][COR_STEP:] \
    .reset_index(drop=True) \
    .add_suffix('_future')
val = pres.join(future)
val.corr()

From this (and some other locations we check), we can see that the correlation of all the variables to occupy is moderately high. (It might not look that high, but considering that we can use multiple steps of lag features, it is a pretty good correlation.) In addition, it is also nice to see that the correlation of `flow` and `speed` to `occupy` is also high, which suggests that the variables might be linearly correlated and could be used to predict each other.

In [ ]:
def plot_periodogram(ts, detrend='linear', ax=None):
    """
    Plots the periodogram of a time series.

    Args:
        ts (pd.Series): A time series.
        detrend (str): Detrending method for the time series.
        ax (matplotlib.axes.Axes): The axes on which to plot.

    Returns:
        ax (matplotlib.axes.Axes): The axes on which the periodogram is plotted.
    """
    fs = pd.Timedelta(weeks=4) / pd.Timedelta(minutes=5)
    frequencies, spectrum = periodogram(
        ts,
        fs=fs,
        detrend=detrend,
        window="boxcar",
        scaling='spectrum',
    )
    if ax is None:
        _, ax = plt.subplots()
        
    ax.step(frequencies, spectrum, color="purple")
    ax.set_xscale("log")
    ax.set_xticks([4, 30, 30*24])
    ax.set_xticklabels(
        [
            "Weekly",
            "Daily",
            "Hourly"
        ],
        rotation=30,
    )
    ax.ticklabel_format(axis="y", style="sci", scilimits=(0, 0))
    ax.set_ylabel("Variance")
    ax.set_title("Periodogram")
    return ax

plot_periodogram(location_0["occupy"])

Here, we graph a [periodogram](https://en.wikipedia.org/wiki/Periodogram) to test the seasonality of the `occupy` variable. The periodogram shows the strength of the frequencies present in the time series, and can be used to identify the dominant frequencies or periodicities in the data. From the graph above, it suggests that there seem to be a pattern in the occupy variable daily, which makes sense, since many people work and travel on a fixed schedule. (for example busy hours after work will have higher traffic) For this reason, we will add an `hour` feature on prediction.

As said earlier, the number of rows in this is pretty large (and when we tried add >10 lag features, the memory limit exceeded >8gb). So, we will instead be working on hourly timesteps instead of 5 minutes interval. The data within an hour (12 5-minute timesteps) will be averaged.

In [ ]:
location_0["hour"] = ((location_0["timestep"] - 1) // 12)
grouped = location_0.groupby("hour").mean().reset_index()
grouped.head()

In [ ]:
grouped["occupy"][:24*5].plot();

In [ ]:
COR_STEP = 12
pres = grouped[['flow', 'occupy', 'speed']][0:-(COR_STEP)].reset_index(drop=True)
future = grouped[['flow', 'occupy', 'speed']][COR_STEP:] \
    .reset_index(drop=True) \
    .add_suffix('_future')
val = pres.join(future)
val.corr()

After merging the timesteps, we get a higher correlation with future values, which is nice.

<a id="section-5"></a>
# Data Preparation

Now that we are done with the EDA, it is time to prepare our data for prediction. We add the features we found useful from the EDA. We also normalize and split our data to train and test datasets.

In [ ]:
# creating 3-dimensional array for [timestep, timeframe, features]
def create_dataset(location, WINDOW_SIZE):
    
    # mask a certain location
    location_current = traffic[traffic["location"]==location].reset_index()
    
    # group to hour and average 12 (5-minute) timesteps
    location_current["hour"] = ((location_current["timestep"] - 1) // 12)
    grouped = location_current.groupby("hour").mean().reset_index()
    
    # add hour features as mod 24 cycle (0...23)
    grouped['day'] = (grouped['hour'] // 24) % 7
    grouped['hour'] %= 24
    
    one_hot_hour = pd.get_dummies(grouped['hour'])
    one_hot_hour = one_hot_hour.add_prefix('hour_')
    
    # merge all the features together to get a total of 27 features
    hour_grouped = pd.concat([grouped[["occupy", "flow", "speed"]], one_hot_hour], axis=1)
    hour_grouped = np.array(hour_grouped)
    
    X, Y = [], []
    
    # add lag features (in reverse time order)
    for i in range(len(hour_grouped) - WINDOW_SIZE):
        X.append(hour_grouped[i:(i + WINDOW_SIZE)][::-1]) # reverse the order
        Y.append(hour_grouped[i + WINDOW_SIZE, 0]) # index 0 is occupy
    
    return X,Y # returns (timestep, timeframe, features) and (target)

We make a `create_dataset` function to make all the features that we need for the prediction (hour features and lag features as multiple dimensions). Each lag step (lag steps total of WINDOW_SIZE) will be included to X in the second dimension (in reverse order). We format the array X as dimension (timestep, timeframe, features) and Y as dimension (target).

In [ ]:
# creating 4-th dimension for the locations
X, Y = [], []

for location in range(170):
    a,b = create_dataset(location, WINDOW_SIZE=24)
    X.append(a)
    Y.append(b)
    
X = np.moveaxis(X,0,-1)
Y = np.moveaxis(Y,0,-1)

print(X.shape)
print(Y.shape)

Finally here, we merge all the locations together to get X `(timestep, timeframe, features, location)` and Y `(timestep, location)`.

In [ ]:
TRAIN_SIZE = 0.8
TEST_SIZE  = 0.2

train_size = int(len(X) * TRAIN_SIZE)
test_size  = int(len(X) * TEST_SIZE)

train_X, train_Y = X[:train_size], Y[:train_size]
test_X, test_Y = X[train_size:], Y[train_size:]

print(train_X.shape)
print(train_Y.shape)
print(test_X.shape)
print(test_Y.shape)

Here, we split the data into train and test with `train : test` ratio of `0.8 : 0.2`. In a time series analysis, we don't usually use random splitting because it wont make sense to predict data in gaps, so we split the first 80% as train while the last 20% is used as test set. 

In [ ]:
scaler_X = MinMaxScaler()
scaler_Y = MinMaxScaler()
train_X = scaler_X.fit_transform(train_X.reshape(train_X.shape[0] * train_X.shape[1], -1)) \
                   .reshape(train_X.shape[0], train_X.shape[1], -1)
test_X = scaler_X.transform(test_X.reshape(test_X.shape[0] * test_X.shape[1], -1)) \
                   .reshape(test_X.shape[0], test_X.shape[1], -1)
train_Y = scaler_Y.fit_transform(train_Y)
test_Y = scaler_Y.transform(test_Y)

Then, we apply a `MinMaxScaler` to scale both X and Y features to the range (0,1). (Note that we need to use a different scaler for X and Y. Also, the good practice is to not fit using test since we assume that it is a fresh data and we have no information about it, so we only using train split to fit the scaler.

In [ ]:
print(train_X.shape)
print(test_X.shape)
print(train_Y.shape)
print(test_Y.shape)

Note that the shape changed a bit for X. I reshaped it so that the dimensions are `(timestep, timeframe, features*location)`. (we merged the locations together so that the final dimensions have features for every 170 locations.)

<a id="section-6"></a>
# Model Training

Finally, the exciting part! We get to see our prediction model in action. We will use an LSTM layer as the input layer while we will output a vector of 170 values, each one to predict the value of `occupy` of each location. 

The reason we used an LSTM layer is because the LSTM is built to remember information from earlier timesteps and gain information from their relation.

We will use `MSE` for the loss and `RMSE` as the metric. (metric will not be used for backpropagation and only serve to look at how well the prediction is at that iteration)

Also, we use `ReLu` since our task is a regression task.

In [ ]:
model = Sequential([
    LSTM(256, return_sequences=True, input_shape=(train_X.shape[1], train_X.shape[2])),
    LSTM(256, return_sequences=False),
    Dropout(0.2),
    Dense(256, activation='relu'),
    Dropout(0.2),
    Dense(170, activation='linear'),
])

model.compile(loss='mse', optimizer='adam', metrics=[RootMeanSquaredError()])

In [ ]:
model.summary()

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model, show_shapes=True, show_layer_names=True)

Once the model architecture is set, it is time to train the model. We can tune the hyperparemeters: epoch (the number of training iterations) and batch_size (the number of rows to include in a single forward and backpropagation). We will include validation split of 0.1 (by default it will take 10% of the last few rows and use it as validation data, which is not used for training but we can use it to evaluate on an unseen data for every given step).

In [ ]:
# Train the model
history = model.fit(train_X, train_Y, epochs=150, batch_size=32, validation_split=0.1, verbose=2)

After fitting our training set with the model, we can evaluate whether our model overfits the training set or not. One way of doing it is by plotting the loss functions at each step of the training.

In [ ]:
def plot_training(training_history, text, width):
    history = training_history.history[text]
    
    # creates a moving average plot to reduce variations
    moving_average = [float("NaN") for i in range(width)]
    for i in range(width, len(history)+1):
        moving_average.append(np.mean(np.array(history[i-width:i+1])))
        
    plt.plot(history)
    plt.plot(moving_average)
    plt.title(text)
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['value','moving average'], loc='upper left')
    plt.show()

In [ ]:
# Plots the loss (MSE and RMSE) of both test and validation data
WIDTH = 10
plot_training(history,'loss',WIDTH)
plot_training(history,'val_loss',WIDTH)
plot_training(history,'val_root_mean_squared_error',WIDTH)
plot_training(history,'root_mean_squared_error',WIDTH)

We can observe that the validation loss value flattening after 120 epochs of training, despite that the training loss keep decreasing. If the training keeps going, our model would overfit to the training set (lower training loss but higher validation loss). In order to avoid those issues, it is best that we would limit the number of training epochs to around that value, which is 150 epochs for this case.

<a id="section-7"></a>
# Evaluate Prediction

Now time to evaluate how good our model is. We add a moving average as the baseline. The reason we need a baseline is to see whether our RMSE score and Spearman score is actually good, not just due to random guessing.

In [ ]:
def predict_and_calc_score(X, Y):
    # prediction of Y using the model
    pred = model.predict(X) 
    # moving average of Y
    window_size = 12
    moving_average = np.apply_along_axis(
        lambda x: np.convolve(x, np.ones(window_size)/window_size, mode='same'), axis=0, arr=Y)
    moving_average = np.concatenate((np.zeros((1, Y.shape[1])), moving_average), axis=0)[:Y.shape[0]]
    
    # scale it back to the original scale
    pred_scaled = scaler_Y.inverse_transform(pred)
    moving_average_scaled = scaler_Y.inverse_transform(moving_average)
    Y_scaled = scaler_Y.inverse_transform(Y)
    
    # calculate the RMSE
    baseline_RMSE = np.sqrt(np.mean((Y_scaled - moving_average_scaled) ** 2))
    model_RMSE = np.sqrt(np.mean((Y_scaled - pred_scaled) ** 2))
    
    return Y_scaled, pred_scaled, moving_average_scaled, model_RMSE, baseline_RMSE

def plot_prediction(actual, prediction, moving_average):
    # Plot the actual values
    plt.plot(actual, label="True value", linestyle='-', linewidth=1, marker='s', markersize=1)
    
    # Plot the moving average
    plt.plot(moving_average, label="Moving Average", linestyle='--', linewidth=1, marker='s', markersize=1)
    
    # Plot the predicted values
    plt.plot(prediction, label="Prediction", linestyle='--', linewidth=1, marker='o', markersize=1)
    
    # Set the title and axis labels
    plt.title('Prediction vs. True Value', fontsize=16)
    plt.xlabel('Hour Timesteps')
    plt.ylabel('Output Value')

    # Add a legend to the plot
    plt.legend(loc='upper left', fontsize=12)
    
    # Display the plot
    plt.show()

In [ ]:
train_actual, train_prediction, train_moving_average, train_RMSE, baseline_RMSE = \
    predict_and_calc_score(train_X, train_Y)

print("Train Moving Average RMSE:", baseline_RMSE)
print("Train Prediction RMSE:", train_RMSE)

corr, pval = stats.spearmanr(train_actual, train_moving_average)
mov_spearman_corr = np.mean(corr)
corr, pval = stats.spearmanr(train_actual, train_prediction)
pred_spearman_corr = np.mean(corr)

print("Train Moving Average Spearman Correlation:", mov_spearman_corr)
print("Train Prediction Spearman Correlation:", pred_spearman_corr)

In [ ]:
location = 3
plot_prediction(train_actual[:,location], train_prediction[:,location], train_moving_average[:,location])

The train RMSE and Spearman is good. It is better than the moving average and shows that the model is not underfitting. However, the true test lies on the test evaluation (since Neural Network models might overfit).

In [ ]:
test_actual, test_prediction, test_moving_average, test_RMSE, baseline_RMSE = \
    predict_and_calc_score(test_X, test_Y)

print("Test Moving Average RMSE:", baseline_RMSE)
print("Test Prediction RMSE:", test_RMSE)

corr, pval = stats.spearmanr(test_actual, test_moving_average)
mov_spearman_corr = np.mean(corr)
corr, pval = stats.spearmanr(test_actual, test_prediction)
pred_spearman_corr = np.mean(corr)

print("Test Moving Average Spearman Correlation:", mov_spearman_corr)
print("Test Prediction Spearman Correlation:", pred_spearman_corr)

In [ ]:
location = 10
plot_prediction(test_actual[:,location], test_prediction[:,location], test_moving_average[:,location])

The test RMSE and Spearman is also good! It is better than the moving average and shows that the model is not overfitting and can generalize well to new data.

In [ ]:
model.save('LSTM_model')

In [ ]:
!zip -r LSTM_Model.zip /kaggle/working/LSTM_model

Finally, we can also save the model.

<a id="section-8"></a>
# Conclusion

To conclude, we have made a predictive model using an LSTM neural network. We included hour and lag features, and also carefully scale and split the data. In the end, our model was able to predict both the train and test data better than our baseline which is moving average of previous values. We believe that this model can be further optimized and tested to help solve one of the real world issues, which is traffic management.